# Modeling Notebook

This notebook will contain model iterations as well as continued data exploration. I will attempt a variety of machine learning methods as well as different feature combinations and engineering. I also intend to experiment with different imputation methods and hope to create a custom imputation objectAfter actually exploring the data in the 04_explore_data I feel as though I have a much better idea of what I am working with.

### Plan
- Import data and do initial cleaning
- Inital model iterations with IterativeImputer (experiment with different models)
- Feature experimentation/engineering
- Imputation Object?

In [19]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Scaling
from sklearn.preprocessing import StandardScaler

# Data Split/Cross Validation
from sklearn.model_selection import train_test_split, cross_val_score, KFold, GridSearchCV

# Model Metrics
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import r2_score

# Models
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor

# Ensembles
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

# Use functions from .py file
%load_ext autoreload
%autoreload 2
import os
import sys
module_path = os.path.abspath(os.path.join(os.pardir, os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
    
import src.data_gathering as dg

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Import Data and Clean

In [26]:
energy = dg.energy_data()
weather = dg.weather_data()

Drop rows from weather dataframe at time 23:59:00 from each day. 

In [27]:
to_drop = weather[(weather.index.hour == 23) & (weather.index.minute == 59)].index

In [28]:
weather.drop(to_drop, axis=0, inplace=True)

Convert all instances of \* in dataframe to np.nan and all instances of '' in wind direction column to np.nan as well

In [29]:
weather['HourlyDewPointTemperature'].loc[weather['HourlyDewPointTemperature'] == '*'] = np.nan
weather['HourlyDryBulbTemperature'].loc[weather['HourlyDryBulbTemperature'] == '*'] = np.nan
weather['HourlyRelativeHumidity'].loc[weather['HourlyRelativeHumidity'] == '*'] = np.nan
weather['HourlyVisibility'].loc[weather['HourlyVisibility'] == '*'] = np.nan

In [30]:
weather['HourlyWindDirection'] = weather['HourlyWindDirection'].replace('', np.nan)

Convert weather dataframe to float

In [31]:
weather = weather.astype(float)

Now I can begin to do some modeling.

## Initial Model Iterations

My initial MVP model iterations will use IterativeImputer to impute the missing values. In later iterations I hope to create a custom imputation object that can incorporate PCA as well because there is a lot of multicollinearity between weather features, but for now this will work.

In [32]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [33]:
imp = IterativeImputer(random_state=42)

In [34]:
weather_iter_imputed = imp.fit_transform(weather)

The IterativeImputer object returns a numpy array, so I need to put this data back into a dataframe using the columns and index from the original weather dataframe.

In [35]:
weather_df = pd.DataFrame(index=weather.index, columns=weather.columns, data=weather_iter_imputed)

Now that I have all of the weather and energy data, I can combine the two into a single dataframe. Since I am modeling energy production three hours in advance, I will incorporate a three hour lag between the weather data and the solar energy output. Before I do that I need to aggregate the weather dataframe into hourly format. To ensure the data lines up correctly (energy output and weather conditions from three hours prior) I am going to add a column to the weather dataframe that gives says the time of weather observation. Once I am certain everything is formatted correctly, I can drop this column

In [346]:
weather_hourly = weather_df.resample('H').mean()

In [347]:
weather_hourly['time'] = weather_hourly.index

In [39]:
base_df = pd.concat([energy, weather_hourly.shift(3)], axis=1)

In [40]:
base_df.head()

,nexus_meter,HourlyAltimeterSetting,HourlyDewPointTemperature,HourlyDryBulbTemperature,HourlyRelativeHumidity,HourlyStationPressure,HourlyVisibility,HourlyWindSpeed,HourlyWindDirection,HourlyPrecipitation,cloud_coverage,time
DATE,,,,,,,,,,,,
2018-01-29 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
2018-01-29 01:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
2018-01-29 02:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
2018-01-29 03:00:00,NaN,30.3425,28.0,30.0,92.25,29.5125,5.75,14.25,327.5,0.017317,80.0,2018-01-29 00:00:00
2018-01-29 04:00:00,NaN,30.3400,28.0,30.0,92.00,29.5100,3.00,10.00,340.0,0.000000,80.0,2018-01-29 01:00:00


Everything seems to be formatted correctly. The first 24 rows of the dataframe do not contain any energy data, so I am going to drop those rows. As well as the time column that I created.

In [41]:
to_drop2 = base_df[:'2018-01-29'].index

In [42]:
base_df.drop(to_drop2, axis=0, inplace=True)

In [48]:
base_df.drop('time', axis=1, inplace=True)

In [49]:
base_df.head()

,nexus_meter,HourlyAltimeterSetting,HourlyDewPointTemperature,HourlyDryBulbTemperature,HourlyRelativeHumidity,HourlyStationPressure,HourlyVisibility,HourlyWindSpeed,HourlyWindDirection,HourlyPrecipitation,cloud_coverage
DATE,,,,,,,,,,,
2018-01-30 00:00:00,0.0,30.47,15.0,19.0,84.0,29.64,10.0,6.0,310.0,0.0,0.0
2018-01-30 01:00:00,0.0,30.48,13.0,17.0,84.0,29.65,10.0,6.0,320.0,0.0,0.0
2018-01-30 02:00:00,0.0,30.48,13.0,17.0,84.0,29.65,10.0,5.0,300.0,0.0,0.0
2018-01-30 03:00:00,0.0,30.46,14.0,17.0,88.0,29.63,10.0,6.0,310.0,0.0,0.0
2018-01-30 04:00:00,0.0,30.47,11.0,15.0,84.0,29.64,10.0,6.0,320.0,0.0,0.0


In [50]:
base_df.isna().sum()

nexus_meter                   0
HourlyAltimeterSetting       10
HourlyDewPointTemperature    10
HourlyDryBulbTemperature     10
HourlyRelativeHumidity       10
HourlyStationPressure        10
HourlyVisibility             10
HourlyWindSpeed              10
HourlyWindDirection          10
HourlyPrecipitation          10
cloud_coverage               10
dtype: int64

So there are ten rows that do not contain any weather data, I want to see what at what time index those rows occur, because I may just be able to drop them

In [51]:
base_df[base_df['HourlyAltimeterSetting'].isna() == True]

,nexus_meter,HourlyAltimeterSetting,HourlyDewPointTemperature,HourlyDryBulbTemperature,HourlyRelativeHumidity,HourlyStationPressure,HourlyVisibility,HourlyWindSpeed,HourlyWindDirection,HourlyPrecipitation,cloud_coverage
DATE,,,,,,,,,,,
2018-04-12 03:00:00,-38.90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-04-12 04:00:00,-38.63,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-05-16 03:00:00,-44.90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-05-16 04:00:00,-44.60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-07-25 01:00:00,-53.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-07-25 02:00:00,-52.90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-07-25 03:00:00,-51.60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-07-25 04:00:00,-51.60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-02-20 02:00:00,-50.60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Okay great, they are all at times where no energy data is produced. This brings me to my next point, I am going to focus my analysis on the time of day where energy is most likely to be produced, for now we will consider this 5am to 9pm. Although in Winter these times are unlikely to produce any solar energy, I do not want to miss any energy production in the Spring, Summer, and Fall months.

In [52]:
model_df = base_df[(base_df.index.hour >= 5) & (base_df.index.hour <= 21)]

In [54]:
model_df.head(20)

,nexus_meter,HourlyAltimeterSetting,HourlyDewPointTemperature,HourlyDryBulbTemperature,HourlyRelativeHumidity,HourlyStationPressure,HourlyVisibility,HourlyWindSpeed,HourlyWindDirection,HourlyPrecipitation,cloud_coverage
DATE,,,,,,,,,,,
2018-01-30 05:00:00,0.000,30.47,11.0,14.0,88.0,29.64,10.0,5.0,330.000000,0.0,0.0
2018-01-30 06:00:00,0.000,30.46,11.0,14.0,88.0,29.63,10.0,5.0,350.000000,0.0,0.0
2018-01-30 07:00:00,0.000,30.45,10.0,13.0,88.0,29.62,10.0,0.0,0.000000,0.0,0.0
2018-01-30 08:00:00,0.000,30.47,11.0,14.0,88.0,29.64,10.0,6.0,260.000000,0.0,0.0
2018-01-30 09:00:00,0.000,30.50,11.0,13.0,92.0,29.67,10.0,6.0,260.000000,0.0,0.0
2018-01-30 10:00:00,276.600,30.48,13.0,16.0,88.0,29.65,10.0,0.0,0.000000,0.0,0.0
2018-01-30 11:00:00,10128.000,30.53,13.0,21.0,71.0,29.70,10.0,3.0,166.892929,0.0,0.0
2018-01-30 12:00:00,17570.000,30.52,12.0,23.0,63.0,29.69,10.0,3.0,220.000000,0.0,0.0
2018-01-30 13:00:00,16156.000,30.48,12.0,25.0,58.0,29.65,10.0,0.0,0.000000,0.0,0.0


Great, the first modeling dataframe is all set up, now I can perform a train test split and begin modeling.

In [55]:
model_df.shape

(15351, 11)

I only have about 15,000 observations to work with and I need to split this data into three groups: training, validation, and testing. I would like the majority of data to be used to train the model (at least 10,000 observations). So I am going to do a 70-15-15 split. That gives me about 10,745 data points for training, and about 2,300 for validation and testing

In [62]:
# Separate target and features

# target
y = model_df['nexus_meter']

# features
X = model_df.drop('nexus_meter', axis=1)


In [63]:
# first train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=.15)

I want to make sure X_test has the right amount of observations.

In [64]:
len(X_test)

2303

Great now I can perform the second split for training and validation data

In [83]:
X_t, X_v, y_t, y_v = train_test_split(X_train, y_train, random_state=42, test_size=.18)

I have all of my data separated into train, validation, and testing data. So now I can begin modeling.

## Model Iterations

### Linear Regression

The first model I am going to start with is just a step above the DummyRegressor used for the baseline model. 

### Note
I think past weather conditions will have a beneficial affect on the model. Shift the features back further. I'm thinking if the model can pick up on how the weather is changing and the rate it which it is changing it will likely be able to better infer what the whether WILL be at the time of energy generation.

I need to scale the data before i can train the model. I will use StandardScaler

In [86]:
scaler = StandardScaler()

In [87]:
X_t_scaled = scaler.fit_transform(X_t)

In [88]:
lr = LinearRegression(normalize=False)

In [90]:
# fit on scaled training data
lr.fit(X_t_scaled, y_t)

LinearRegression()

In [91]:
# training predictions
lr_train_preds = lr.predict(X_t_scaled)

The main metric I will be using is RMSE. To gauge this metric, I need to have an idea of what the mean and standard deviation of the target variable is, I will use the variable 'y', which hold's all target data to compute this. I will also be using adjusted R2 to estimate the amount of target variance explained by the features while adjusting the score for number of features I have.

In [92]:
target_mean = np.mean(y)
target_std = np.std(y)
print('Target mean: {}\nTarget std: {}'.format(target_mean, target_std))

Target mean: 4564.371307797537
Target std: 5625.247166565416


In [94]:
np.sqrt(mse(y_t, lr_train_preds))

5041.827998601696

Now I want to check the validation predictions to see how the model fit. I doubt the model is overfit, but I would like to check anyways. First I need to transform the validation data.

In [95]:
X_v_scaled = scaler.transform(X_v)

In [96]:
lr_val_preds = lr.predict(X_v_scaled)

In [97]:
np.sqrt(mse(y_v, lr_val_preds))

5126.357458453091

In [98]:
r2_score(y_t, lr_train_preds)

0.19480069267237066

## Decision Tree Regressor

The next model i would like to test is a decision tree regressor. Along with it being a slightly more complex model than linear regression, it provides more information about the results. Using Decision Tree I will be able to see which feature's the model is considering the most important and splitting on earlier. The training and validation data is already scaled with StandardScaler so i can begin with instantiating a decision tree model and performing cross-validation with cross_val_score. In future model iterations I plan on using KFold so I can experiment with a few different imputation methods within the cross-validation.

In [100]:
# instantiate decision tree regressor with default parameters.
dt = DecisionTreeRegressor(random_state=42)

In [101]:
# fit to training data
dt.fit(X_t_scaled, y_t)

DecisionTreeRegressor(random_state=42)

Now I can use cross_val_score to perform a cross validation over five splits. I still want to focus on RMSE, so I need to create an MSE scorer object to be passed into cross_val_score, since it's default metric is r2. Since I want RMSE, I will take the square root of the mean MSE over five splits.

In [104]:
from sklearn.metrics import make_scorer

In [105]:
mse_score = make_scorer(mse)

In [107]:
np.sqrt(cross_val_score(dt, X_t_scaled, y_t, scoring=mse_score, cv=5).mean())

6329.506983473418

The base DecisionTreeRegressor performed worse than the base LinearRegression model. I am going to instantiate another DT regression object and change a few of the parameters to try and produce a better performing model. To start I will change min_samples_leaf from the default of 1 to 25. My thinking behind this is that the Decision Tree is probably splitting the data too much, energy production isn't going to have too big of a fluctuation with a minor change in one of the features, so by grouping results together in leaf nodes it should produce better predictions.

In [120]:
dt2 = DecisionTreeRegressor(random_state=42, min_samples_leaf=25)

In [121]:
np.sqrt(cross_val_score(dt2, X_t_scaled, y_t, scoring=mse_score, cv=5).mean())

5075.467944015973

Changing that single parameter brought the RMSE down by over 1,000. Next I want to change min_samples_split from 2 to 10, for the same reason discussed above.

In [136]:
dt3 = DecisionTreeRegressor(random_state=42, min_samples_leaf=25, min_samples_split=10)

In [137]:
# training RMSE cross val score over 5 splits
np.sqrt(cross_val_score(dt3, X_t_scaled, y_t, scoring=mse_score, cv=5).mean())

5075.467944015973

That did not change the score too much. Those two parameters are pretty similar to one another, so i am going to try changing max_depth. Again, I don't want to model splitting the energy production into two seperate leaf nodes due to a very minor difference in one of the features, because a minor change in a single weather condition will not have a very large affect on solar energy production. I want to see how deep the current decision tree is before changing the max_depth parameter.

In [138]:
dt3.fit(X_t_scaled, y_t)

DecisionTreeRegressor(min_samples_leaf=25, min_samples_split=10,
                      random_state=42)

In [145]:
print(dt3.tree_.max_depth)

16


The current depth of the tree is 16. I am going to see if 10 will reduce the error.

## NOTE
Time of year and time of day are both going to influence how much energy is produced, add a feature for both of these

In [152]:
dt4 = DecisionTreeRegressor(random_state=42, max_depth=10, min_samples_leaf=25)

In [153]:
np.sqrt(cross_val_score(dt4, X_t_scaled, y_t, scoring=mse_score, cv=5).mean())

5058.020095469636

That reduced the RMSE score slightly. I want to see how this tree is fitting to the data (overfit or underfit) by checking it's performance on the validation data

In [154]:
dt4.fit(X_t_scaled, y_t)

DecisionTreeRegressor(max_depth=10, min_samples_leaf=25, random_state=42)

In [155]:
np.sqrt(mse(y_v, dt4.predict(X_v_scaled)))

5121.5222838481895

The RMSE of the validation data is only slightly higher than that of the training data which tells me the model's variance is not too high and it can generalize fairly well to unseen data. Now I would like to experiment with a few more complex ensemble methods: random forest and gradient boosting. After i have worked with these models I will want to experiment with a few different imputation methods and do some feature engineering.

## Random Forest

The first ensemble I want to work with is random forest. I need to keep in mind the changes to the hyperparameters I made for the Decision Tree model above, because those changes should have the same effect to this model. I want to start by creating a random forest model with a max_depth of 15, min_samples_leaf of 25 and 100 estimators.

In [166]:
rf1 = RandomForestRegressor(random_state=42, max_depth=15, min_samples_leaf=25, n_estimators=100)

In [167]:
# Cross val RMSE score over 5 splits
np.sqrt(cross_val_score(rf1, X_t_scaled, y_t, scoring=mse_score, cv=5).mean())

4820.799769697813

The random forest model with similar parameters to the decision tree above performed a little better, this is to be expected because RandomForest is an ensemble which usually perform better than model's that are not ensemble methods. RandomForest does have quite a few more hyperparameters i can experiment with. The next iteration I want to change max_features to .5, this will make the model consider only half of the features for each split, and should help prevent a tree from splitting due to a minor change in one of the features.

In [182]:
rf2 = RandomForestRegressor(random_state=42, max_depth=15, min_samples_leaf=25, n_estimators=100, max_features=.5)

In [183]:
# Cross val RMSE score over 5 splits
np.sqrt(cross_val_score(rf2, X_t_scaled, y_t, scoring=mse_score, cv=5).mean())

4830.154086478536

That iteration performed slightly worse. I am going to leave max_features alone and focus on how the model is splitting the nodes. The next iteration i want to set max_leaf_nodes to 25, min_samples_split to 25, and ignore min_samples_leaf

In [190]:
rf3 = RandomForestRegressor(random_state=42, max_depth=15, min_samples_split=25, max_leaf_nodes=25)

In [191]:
# Cross val RMSE score over 5 splits
np.sqrt(cross_val_score(rf3, X_t_scaled, y_t, scoring=mse_score, cv=5).mean())

4934.239585270432

That brought the RMSE score up again. I think the best way to improve the performance of these models would be to add some features.

## Feature Engineering

For starters I want to incorporate the time of year and time of day as a feature, as this will likely play a big role in how much energy is produced. For time of year I want to add a feature that gives a numerical representation of the week of the year, and for a time of day I will add a feature that gives the hour the predictions are being made for.

In addition to those two, I want to incorporate more weather data instead of only using the weather conditions at the time of prediction. My thinking is that if the model can pick up on how the weather has changed over the past few hours, and the rate of that change, it will be able to better predict what the weather will be like in the future (three hours).

Since I am forecasting energy prediction in the future, I think it would make more sense to include what time of the day it will be at the point of energy production, so I am going to add the 'hour' column to the energy dataframe.

In [209]:
energy['hour'] = energy.index.hour

In [210]:
energy.head()

,nexus_meter,hour
time,,
2018-01-30 00:00:00,4.0,0
2018-01-30 01:00:00,4.0,1
2018-01-30 02:00:00,4.0,2
2018-01-30 03:00:00,4.0,3
2018-01-30 04:00:00,4.0,4


Same goes for the week. Although the week of energy prediction will be the same week as weather conditions, I want to keep the time features consistent.

In [218]:
energy['week'] = energy.index.week

In [220]:
energy.head()

,nexus_meter,hour,week
time,,,
2018-01-30 00:00:00,4.0,0,5
2018-01-30 01:00:00,4.0,1,5
2018-01-30 02:00:00,4.0,2,5
2018-01-30 03:00:00,4.0,3,5
2018-01-30 04:00:00,4.0,4,5


Great now I have the week of the year and hour of the day for each energy prediction. The next features I would like to create are more weather features. Currently, the only weather conditions I am using to predict solar energy production three hours in the future are the current weather observations (ie the weather conditions three hours prior to energy production). I think by including the weather conditions prior to the current observations, and the rate at which they changed will allow the model to pick up on how the weather may change in the future. In other words, I will use the current weather conditions, weather conditions from three hours prior, and the derivative of each weather condition over that time period to guage the rate at which they are changing to predict solar energy output three hours in advance. This means i will need to create two more weather dataframes.

The first weather dataframe I need to create will be similar to how I structured the base model dataframe. I just need to incorporate a three hour lag and store it in a new dataframe.

In [348]:
weather_lagged_df = weather_hourly.shift(3)

In [349]:
weather_lagged_df.head()

,HourlyAltimeterSetting,HourlyDewPointTemperature,HourlyDryBulbTemperature,HourlyRelativeHumidity,HourlyStationPressure,HourlyVisibility,HourlyWindSpeed,HourlyWindDirection,HourlyPrecipitation,cloud_coverage,time
DATE,,,,,,,,,,,
2018-01-29 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
2018-01-29 01:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
2018-01-29 02:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
2018-01-29 03:00:00,30.3425,28.0,30.0,92.25,29.5125,5.75,14.25,327.5,0.017317,80.0,2018-01-29 00:00:00
2018-01-29 04:00:00,30.3400,28.0,30.0,92.00,29.5100,3.00,10.00,340.0,0.000000,80.0,2018-01-29 01:00:00


In [350]:
weather_hourly.head()

,HourlyAltimeterSetting,HourlyDewPointTemperature,HourlyDryBulbTemperature,HourlyRelativeHumidity,HourlyStationPressure,HourlyVisibility,HourlyWindSpeed,HourlyWindDirection,HourlyPrecipitation,cloud_coverage,time
DATE,,,,,,,,,,,
2018-01-29 00:00:00,30.3425,28.0,30.0,92.25,29.5125,5.75,14.25,327.5,0.017317,80.0,2018-01-29 00:00:00
2018-01-29 01:00:00,30.3400,28.0,30.0,92.00,29.5100,3.00,10.00,340.0,0.000000,80.0,2018-01-29 01:00:00
2018-01-29 02:00:00,30.3300,28.0,30.0,92.00,29.5000,5.00,13.00,340.0,0.000000,80.0,2018-01-29 02:00:00
2018-01-29 03:00:00,30.3300,28.0,30.0,92.00,29.5000,6.00,13.00,320.0,0.000000,80.0,2018-01-29 03:00:00
2018-01-29 04:00:00,30.3300,28.0,30.0,92.50,29.5000,6.00,10.50,325.0,0.009066,80.0,2018-01-29 04:00:00


In [352]:
weather_lagged_df.drop(weather_lagged_df.iloc[0:3].index, axis=0, inplace=True)
weather_hourly.drop(weather_hourly.iloc[0:3].index, axis=0, inplace=True)

Now the dataframe weather_lagged_df holds the weather observations three hour's prior to the time index. I can assure this by investigating the 'time' column in the dataframe. Each value of 'time' is three hours earlier than the index of that row. The next dataframe I need to construct is the rate at which these observations changed over the three hour period. To do this I will find the slope of the value of each weather observation from the past weather observation to current weather observation with the following formula: (current_value-past_value)/3. I use three in the denominator because the change in x (change in time) remains constant.

In [353]:
weather_change_df = pd.DataFrame(index=weather_hourly.index, columns=weather_hourly.columns)

In [354]:
weather_change_df.head()

,HourlyAltimeterSetting,HourlyDewPointTemperature,HourlyDryBulbTemperature,HourlyRelativeHumidity,HourlyStationPressure,HourlyVisibility,HourlyWindSpeed,HourlyWindDirection,HourlyPrecipitation,cloud_coverage,time
DATE,,,,,,,,,,,
2018-01-29 03:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-29 04:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-29 05:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-29 06:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-29 07:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Now i need to iterate through both the weather_hourly dataframe and weather_lagged_df dataframe and subtract the value of the lagged df from the normal df and divide by three. Before I do that I need to drop the first three rows from each dataframe so the calculation will work. These rows would be dropped when combining with the energy data anyways so I am not losing any valuable data

In [355]:
weather_lagged_df.head()

,HourlyAltimeterSetting,HourlyDewPointTemperature,HourlyDryBulbTemperature,HourlyRelativeHumidity,HourlyStationPressure,HourlyVisibility,HourlyWindSpeed,HourlyWindDirection,HourlyPrecipitation,cloud_coverage,time
DATE,,,,,,,,,,,
2018-01-29 03:00:00,30.3425,28.0,30.0,92.25,29.5125,5.75,14.25,327.5,0.017317,80.0,2018-01-29 00:00:00
2018-01-29 04:00:00,30.3400,28.0,30.0,92.00,29.5100,3.00,10.00,340.0,0.000000,80.0,2018-01-29 01:00:00
2018-01-29 05:00:00,30.3300,28.0,30.0,92.00,29.5000,5.00,13.00,340.0,0.000000,80.0,2018-01-29 02:00:00
2018-01-29 06:00:00,30.3300,28.0,30.0,92.00,29.5000,6.00,13.00,320.0,0.000000,80.0,2018-01-29 03:00:00
2018-01-29 07:00:00,30.3300,28.0,30.0,92.50,29.5000,6.00,10.50,325.0,0.009066,80.0,2018-01-29 04:00:00


In [356]:
weather_hourly.head()

,HourlyAltimeterSetting,HourlyDewPointTemperature,HourlyDryBulbTemperature,HourlyRelativeHumidity,HourlyStationPressure,HourlyVisibility,HourlyWindSpeed,HourlyWindDirection,HourlyPrecipitation,cloud_coverage,time
DATE,,,,,,,,,,,
2018-01-29 03:00:00,30.33,28.0,30.00,92.00,29.50,6.0,13.0,320.0,0.000000,80.0,2018-01-29 03:00:00
2018-01-29 04:00:00,30.33,28.0,30.00,92.50,29.50,6.0,10.5,325.0,0.009066,80.0,2018-01-29 04:00:00
2018-01-29 05:00:00,30.34,27.0,30.00,88.00,29.51,10.0,14.0,340.0,-0.001126,80.0,2018-01-29 05:00:00
2018-01-29 06:00:00,30.36,26.0,29.00,89.00,29.53,5.0,16.0,350.0,0.000000,80.0,2018-01-29 06:00:00
2018-01-29 07:00:00,30.37,25.0,28.25,86.75,29.54,9.0,14.5,352.5,-0.000203,80.0,2018-01-29 07:00:00


Now I can iterate through the dataframe and calculate the slope of each weather condition over the three hour time period.

In [357]:
for idx, row in weather_change_df.iterrows():
    weather_change_df.loc[idx] = (weather_hourly.loc[idx] - weather_lagged_df.loc[idx])/3

In [358]:
weather_change_df.head()

,HourlyAltimeterSetting,HourlyDewPointTemperature,HourlyDryBulbTemperature,HourlyRelativeHumidity,HourlyStationPressure,HourlyVisibility,HourlyWindSpeed,HourlyWindDirection,HourlyPrecipitation,cloud_coverage,time
DATE,,,,,,,,,,,
2018-01-29 03:00:00,-0.00416667,0,0,-0.0833333,-0.00416667,0.0833333,-0.416667,-2.5,-0.00577225,0,0 days 01:00:00
2018-01-29 04:00:00,-0.00333333,0,0,0.166667,-0.00333333,1,0.166667,-5,0.00302193,0,0 days 01:00:00
2018-01-29 05:00:00,0.00333333,-0.333333,0,-1.33333,0.00333333,1.66667,0.333333,0,-0.000375227,0,0 days 01:00:00
2018-01-29 06:00:00,0.01,-0.666667,-0.333333,-1,0.01,-0.333333,1,10,0,0,0 days 01:00:00
2018-01-29 07:00:00,0.0133333,-1,-0.583333,-1.91667,0.0133333,1,1.33333,9.16667,-0.00308974,0,0 days 01:00:00


Great, now I can combine all four dataframes into one and begin modeling again. Before I do that I want to change the column names of the dataframes so i can differentiate between the different time lags and change over time.

In [359]:
energy.drop(energy.iloc[0:3].index, axis=0, inplace=True)

In [360]:
weather_hourly.columns

Index(['HourlyAltimeterSetting', 'HourlyDewPointTemperature',
       'HourlyDryBulbTemperature', 'HourlyRelativeHumidity',
       'HourlyStationPressure', 'HourlyVisibility', 'HourlyWindSpeed',
       'HourlyWindDirection', 'HourlyPrecipitation', 'cloud_coverage', 'time'],
      dtype='object')

In [362]:
weather_hourly.drop('time', axis=1, inplace=True)
weather_lagged_df.drop('time', axis=1, inplace=True)
weather_change_df.drop('time', axis=1, inplace=True)

In [363]:
weather_hourly.columns = ['altimeter', 'dew_point', 'temperature', 'humidity', 'station_pressure', 'visibility', 'wind_speed', 'wind_direction', 'precipitation', 'cloud_coverage']
weather_lagged_df.columns = ['lag3_altimeter', 'lag3_dew_point', 'lag3_temperature', 'lag3_humidity', 'lag3_station_pressure', 'lag3_visibility', 
                            'lag3_wind_speed', 'lag3_wind_direction', 'lag3_precipitation', 'lag3_cloud_coverage']
weather_change_df.columns = ['altimeter_change', 'dew_point_change', 'temperature_change', 'humidity_change', 'station_pressure_change', 'visibility_change', 
                            'wind_speed_change', 'wind_direction_change', 'precipitation_change', 'cloud_coverage_change']

In [364]:
weather_change_df = weather_change_df.astype(float)

In [384]:
base_df2 = pd.concat([energy, weather_hourly.shift(3), weather_lagged_df.shift(3), weather_change_df.shift(3)], axis=1)

In [385]:
base_df2.head()

,nexus_meter,hour,week,altimeter,dew_point,temperature,humidity,station_pressure,visibility,wind_speed,...,altimeter_change,dew_point_change,temperature_change,humidity_change,station_pressure_change,visibility_change,wind_speed_change,wind_direction_change,precipitation_change,cloud_coverage_change
DATE,,,,,,,,,,,,,,,,,,,,,
2018-01-29 03:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-29 04:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-29 05:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-29 06:00:00,NaN,NaN,NaN,30.33,28.0,30.0,92.0,29.5,6.0,13.0,...,-0.004167,0.0,0.0,-0.083333,-0.004167,0.083333,-0.416667,-2.5,-0.005772,0.0
2018-01-29 07:00:00,NaN,NaN,NaN,30.33,28.0,30.0,92.5,29.5,6.0,10.5,...,-0.003333,0.0,0.0,0.166667,-0.003333,1.000000,0.166667,-5.0,0.003022,0.0


The first day of observations does not have any energy data, so i will drop the rows up to '2018-01-30'

In [386]:
base_df2.drop(base_df2[:'2018-01-30'].index, axis=0, inplace=True)

Again, I only want to focus on the times that are likely to produce solar energy, for now I will consider this the time period 5am - 9pm each day. I will now subset this dataframe to only include those times.

In [387]:
model_df2 = base_df2[(base_df2.index.hour >= 5) & (base_df2.index.hour <= 21)]

In [388]:
model_df2.head()

,nexus_meter,hour,week,altimeter,dew_point,temperature,humidity,station_pressure,visibility,wind_speed,...,altimeter_change,dew_point_change,temperature_change,humidity_change,station_pressure_change,visibility_change,wind_speed_change,wind_direction_change,precipitation_change,cloud_coverage_change
DATE,,,,,,,,,,,,,,,,,,,,,
2018-01-31 05:00:00,4.0,5.0,5.0,29.99,20.0,30.0,66.0,29.17,10.0,21.0,...,-0.020000,0.333333,0.000000,0.666667,-0.020000,0.0,-0.333333,3.333333,0.0,0.0
2018-01-31 06:00:00,4.0,6.0,5.0,29.93,20.0,30.0,66.0,29.11,10.0,23.0,...,-0.030000,0.333333,0.000000,0.666667,-0.030000,0.0,0.333333,0.000000,0.0,0.0
2018-01-31 07:00:00,4.0,7.0,5.0,29.87,21.0,31.0,67.0,29.05,10.0,24.0,...,-0.046667,0.333333,0.333333,0.333333,-0.046667,0.0,0.666667,0.000000,0.0,0.0
2018-01-31 08:00:00,4.0,8.0,5.0,29.87,21.0,31.0,67.0,29.05,10.0,23.0,...,-0.040000,0.333333,0.333333,0.333333,-0.040000,0.0,0.666667,3.333333,0.0,0.0
2018-01-31 09:00:00,4.0,9.0,5.0,29.87,21.0,31.0,67.0,29.05,10.0,20.0,...,-0.020000,0.333333,0.333333,0.333333,-0.020000,0.0,-1.000000,3.333333,0.0,0.0


The dataframe looks like it is formatted correctly, now I can proceed with more model iterations with the new features. 

## NOTE
There was already a lot of multicollinearity between the original weather conditions, but now there is likely to be a lot more (the weather from three hours ago is highly influential of the weather conditions now), so I will definitely need to perform some dimensionality reduction with PCA during these model iterations.

## Continued Model Iterations

The first thing I need to do is perform another train test split into training, validation, and testing data with the new model dataframe.

In [389]:
# separate target and features

# target
y2 = model_df2['nexus_meter']

# features
X2 = model_df2.drop('nexus_meter', axis=1)

In [391]:
X2_imputed = imp.fit_transform(X2)

In [392]:
X2_df = pd.DataFrame(index=X2.index, columns=X2.columns, data=X2_imputed)

In [393]:
# initial train test split into training and testing data
X_train2, X_test2, y_train2, y_test2 = train_test_split(X2_df, y2, random_state=42, test_size=.15)

In [394]:
# second split into training and validation data
X_t2, X_v2, y_t2, y_v2 = train_test_split(X_train2, y_train2, random_state=42, test_size=.18)

Now I can scale the training and validation data

In [395]:
X_t2_scaled = scaler.fit_transform(X_t2)
X_v2_scaled = scaler.transform(X_v2)

## Random Forest with more features

During earlier model iterations, random forest was the best performing model I had created, so I will begin with a random forest model on the new feature set. I am using the same hyperparameters as before so I can see how the new features either help (hopefully) or hurt the model.

In [396]:
rf = RandomForestRegressor(random_state=42, max_depth=15, min_samples_leaf=25, n_estimators=100)

Now I can calculate the cross-validation RMSE score over 5 splits

In [397]:
np.sqrt(cross_val_score(rf, X_t2_scaled, y_t2, scoring=mse_score, cv=5).mean())

4.834985733299825

Well that doesn't make sense. I think something might have gone wrong during feature engineering or making the new model_df because I don't think the RMSE should be that low.

In [398]:
rf.fit(X_t2_scaled, y_t2)

RandomForestRegressor(max_depth=15, min_samples_leaf=25, random_state=42)

In [399]:
rf_train_preds = rf.predict(X_t2_scaled)

In [400]:
np.sqrt(mse(y_v2, rf.predict(X_v2_scaled)))

4.180454389728944